In [ ]:
from torch import tensor
import torch

import torch

# Define a type for a named tensor with two dimensions
ImagePlaneDetections = torch.TensorType[("x", float), ("y", float)]


class Tracker:
    def __init__(self, num_particles: int = 1000) -> None:
        self.num_particles = num_particles
        # state: [x, y, z, vx, vy, zv, ax, ay, az, ax, ay, az, weight]
        self.states = torch.randn(
            (num_particles, 3 * 3 + 1, 1),
            names=["x", "y", "z", "vx", "vy", "vz", "ax", "ay", "az", "weight"],
        )

    @staticmethod
    def dynamics(state_prev, dt):
        state_prev = state_prev
        v = v_prev + torch.randn_like(v_prev) * 0.1  # Add noise to the velocity
        x = x_prev + v * dt
        return x, v

    @staticmethod
    def state_to_observation(state, H) -> ImagePlaneDetections:

        return (H @ state).to(ImagePlaneDetections)

    # Define the likelihood function
    @staticmethod
    def likelihood(obs_state: torch.tensor, pred_state):
        sigma = 0.1  # Standard deviation of the observation noise
        p = torch.exp(-0.5 * ((x_obs - x_pred) / sigma) ** 2)
        return p

In [ ]:
# Define the 3D point in homogeneous coordinates
P_hom = np.array([[X], [Y], [Z], [1]], dtype=np.float32)

# Project the 3D point to the image plane using the projection matrix
P_proj_hom = np.dot(P, P_hom)

# Normalize the image coordinates
P_proj = P_proj_hom[:2] / P_proj_hom[2]

# Print the projected point on the image plane
print("Projected point:", P_proj)